In [5]:
import tensorflow as tf
import tensorflow.keras as keras
import pandas as pd

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
print(gpus)

if gpus:
  try:    
    for gpu in gpus:
      tf.config.experimental.set_visible_devices(gpu, 'GPU')
      tf.config.experimental.set_memory_growth(gpu, True)

    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPU")
  except Exception as e:
    print("EXCEPTION")
    print(e)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
2 Physical GPUs, 1 Logical GPU


I0000 00:00:1739072463.965769 3292769 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13665 MB memory:  -> device: 1, name: NVIDIA GeForce RTX 4090, pci bus id: 0000:02:00.0, compute capability: 8.9


In [10]:
df_labels = pd.read_csv('./data/blood-cell/dataset-master/dataset-master/labels.csv')[['Image', 'Category']].dropna()
df_labels

,Image,Category
0,0,NEUTROPHIL
1,1,NEUTROPHIL
2,2,NEUTROPHIL
3,3,NEUTROPHIL
4,4,NEUTROPHIL
...,...,...
405,405,NEUTROPHIL
407,407,NEUTROPHIL
408,408,NEUTROPHIL
409,409,NEUTROPHIL
